In [1]:
import matplotlib.pyplot as plt
import circle_fit as cf
import numpy as np
import cv2
import os
%run ../functions.ipynb

In [2]:
def image_processing(gray_img):
    blur = cv2.GaussianBlur(gray_img, (5,5), 0)
    nlm  = cv2.fastNlMeansDenoising(blur, None, 10, 7, 21)
    _, otsu = cv2.threshold(nlm, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    edge = cv2.Canny(otsu, 0, 20)
    
    return edge, blur

In [3]:
def get_edge_points(edge_img):
    edge_points = [[j,i] for i, r in enumerate(edge_img) for j, c in enumerate(r) if c == 255]

    return edge_points

## 測試擴增Edge 試圖建立虛擬連結

### Version 1 移動有問題的點

In [4]:
def move_edge(edited_edge, edge, img):
    
    edited_index = []
    for index, point in enumerate(edge):
        x, y = point[:]
        value = img[y,x]
        
        for i in range(-1, 2, 1):
            for j in range(-1, 2, 1):
                if int(value) - int(img[y+i,x+j]) > 2:
                    edited_edge.append([x+j, y+i])
                    edited_index.append(index)
    
    edited_edge = np.delete(edited_edge, edited_index, axis=0)
    return edited_edge

### Versoin 2 直接增加可能的點

In [5]:
def add_edge(edited_edge, edge_points, img):
    
    for index, point in enumerate(edge):
        x, y = point[:]
        value = img[y,x]
        
        for i in range(-1, 2, 1):
            for j in range(-1, 2, 1):
                if int(value) - int(img[y+i,x+j]) > 2:
                    edited_edge.append([x+j, y+i])
    
    return edited_edge

### Version 3 直接去除問題點

In [6]:
def del_edge(edited_edge, edge_points, img):
    
    edited_index = []
    for index, point in enumerate(edge):
        x, y = point[:]
        value = img[y,x]
        
        for i in range(-1, 2, 1):
            for j in range(-1, 2, 1):
                if int(value) - int(img[y+i,x+j]) > 2:
                    edited_index.append(index)
    
    edited_edge = np.delete(edited_edge, edited_index, axis=0)
    return edited_edge

## 去除極端

In [7]:
def remove_extreme(points):
    tmp = points[points[:,0].argsort()]
    tmp = tmp[10:-10, :]
    
    tmp = tmp[tmp[:,1].argsort()]
    tmp = tmp[10:-10, :]
    return tmp

## 無差別增加

In [14]:
def add_neighbors(edge):
    tmp = edge.copy()
    
    for index, point in enumerate(edge):
        x, y = point[:]
        
        for i in range(-1, 2, 1):
            for j in range(-1, 2, 1):
                tmp.append([x+j, y+i])
    
    return tmp

## 編輯邊緣

In [15]:
def edit_edge(edge, img, funct):
    edited_edge = edge.copy()
    
    if funct == 0:
        edited_edge = move_edge(edited_edge, edge, img)
    elif funct == 1: 
        edited_edge = add_edge(edited_edge, edge, img)
    else:
        edited_edge = del_edge(edited_edge, edge, img)
    
    edited_edge = np.unique(edited_edge, axis=0)
    return edited_edge

In [21]:
os.chdir("/home/mj/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")

file = "10_mean_80.bmp"
img = cv2.imread(file,0)
img_edge, img_blur = image_processing(img)
edge = get_edge_points(img_edge)

edited_edge = edit_edge(edge, img_blur, funct=0)
edited_edge = remove_extreme(edited_edge)
edited_edge = add_neighbors(edge)
edited_edge = np.unique(edited_edge, axis=0)

print(len(edge))
print(len(edited_edge))

cx,cy,r,_ = cf.least_squares_circle((edge))
print("{}  {}".format(cx, cy))
cx,cy,r,_ = cf.least_squares_circle((edited_edge))
print("{}  {}".format(cx, cy))

442
1514
203.43340624689515  99.48214409869645
203.44549777281  99.49552666649772


In [31]:
os.chdir("/home/mj/Github/NTU/ARCS/img/X-Ray/Case_3/Cut_2")
try:
    
    images = os.listdir()
    images.sort()
    
    predictions = []
    for image in images:
        img = cv2.imread(image,0)
        img_edge, img_blur = image_processing(img)
        edge = get_edge_points(img_edge)
        edited_edge = edit_edge(edge, img_blur, funct=1)
#         edited_edge = remove_extreme(edited_edge)
#         edited_edge = add_neighbors(edge)
#         edited_edge = np.unique(edited_edge, axis=0)
        
        cx,cy,r,_ = cf.least_squares_circle((edited_edge))
        predictions.append([cx, cy])
        print("{}: {}, {}".format(image ,cx, cy))
        
except Exception as e:
    print(e)

10_mean_0.bmp: 181.67687321470495, 96.87066921198652
10_mean_1.bmp: 181.59941065369617, 96.87420776877117
10_mean_10.bmp: 181.62256434043044, 96.83923308575152
10_mean_11.bmp: 181.89170532831955, 96.65249354646441
10_mean_12.bmp: 181.40600409480223, 96.99376840052763
10_mean_13.bmp: 181.6105769964037, 96.7686030064725
10_mean_14.bmp: 181.6109942399868, 97.05185234243683
10_mean_15.bmp: 181.55196428893495, 96.83273308348835
10_mean_16.bmp: 181.74714363356722, 96.88777347460353
10_mean_17.bmp: 181.82304532527, 96.96275856458584
10_mean_18.bmp: 181.66447032500918, 96.96298660560184
10_mean_19.bmp: 181.58057824696238, 96.85101010126763
10_mean_2.bmp: 181.7777721668212, 96.80591038185756
10_mean_20.bmp: 181.56678815924766, 96.94193642996709
10_mean_21.bmp: 181.5832477635625, 96.65998611345911
10_mean_22.bmp: 181.60562961025894, 96.82707535070224
10_mean_23.bmp: 181.62591385969097, 97.01331961957143
10_mean_24.bmp: 181.61592094337476, 97.01870239024483
10_mean_25.bmp: 181.6383028606056, 96.8

In [32]:
show_resoult(predictions)

-----------------------------------------------------
Statistics: 
Mean Centroid: (181.66349, 96.92410)
Variance     : (0.01371, 0.01728)
Max Length   : 0.57504, 0.63329 (pixels)


In [16]:
os.chdir("/home/mj/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")
file_1 = "10_mean_63.bmp"
img_1 = cv2.imread(file_1,0)
img_edge_1, img_blur_1 = image_processing(img_1)
edge_1 = get_edge_points(img_edge_1)
edited_edge_1 = edit_edge(edge_1, img_blur_1, funct=1)

file_2 = "10_mean_63.bmp"
img_2 = cv2.imread(file_2,0)
img_edge_2, img_blur_2 = image_processing(img_2)
edge_2 = get_edge_points(img_edge_2)
edited_edge_2 = edit_edge(edge_2, img_blur_2, funct=1)


h, w = img.shape[:2]
tmp = np.zeros((h,w,3))

for x, y in edited_edge_1:
    tmp[y,x,1] = 255
    
for x, y in edited_edge_2:
    tmp[y,x,2] = 255
    
    
# 顯示圖片
cv2.imshow("Compare", tmp)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(3.4.2) /tmp/build/80754af9/opencv-suite_1535558553474/work/modules/highgui/src/window.cpp:632: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Carbon support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [3]:
a = [[1,2], [3,4]]

for b,c in a:
    print(c)
    

2
4


## 測試多次隨機取子點擬合的分佈
假定平均 cx: 203.61381  cy: 99.17248
### 結論：無法單靠尋找 Fram 各自的 Median 或 多次 Random subsampled 做連結

In [50]:
os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")

file = "10_mean_80.bmp"
img = cv2.imread(file,0)
edge = image_processing(img)
edge_points = get_edge_points(edge)


xs = []
ys = []
for iter in range(10):
    sampled_indices = np.random.choice(range(len(edge_points)), size=int(len(edge_points)), replace=False)
    sample_points = edge_points[sampled_indices]
#     cx,cy,r,_ = cf.least_squares_circle((sample_points))
    cx = np.median(sample_points[:,0])
    cy = np.median(sample_points[:,1])
    xs.append(cx)
    ys.append(cy)

print(xs)
print(ys)

[205.0, 205.0, 205.0, 205.0, 205.0, 205.0, 205.0, 205.0, 205.0, 205.0]
[100.5, 100.5, 100.5, 100.5, 100.5, 100.5, 100.5, 100.5, 100.5, 100.5]
